# Previsão de Probabilidade de Preços por Desvio Padrão

  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.

Defaulting to user installation because normal site-packages is not writeable
     ---------------------------------------- 55.9/55.9 kB 3.0 MB/s eta 0:00:00
  Using cached html5lib-1.1-py2.py3-none-any.whl (112 kB)
  Using cached pytz-2022.7.1-py2.py3-none-any.whl (499 kB)
  Using cached lxml-4.9.2-cp39-cp39-win_amd64.whl (3.9 MB)



ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
conda-repo-cli 1.0.4 requires pathlib, which is not installed.


## 1. Importação das Bibliotecas e Configurações

In [6]:
%config IPCompleter.greedy=True
import yfinance as yf
import datetime
import numpy as np
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots

## 2. Definição de Parâmetros

In [7]:
hoje = datetime.date.today()
inicio = hoje - datetime.timedelta(days = 365) # pegar 1 ano de dados
hoje = str(hoje)
inicio = str(inicio)

dias_venc = 15 #modificar colocando quantos dias faltam para o vencimento da opção ou quantos dias gostaria para o range da previsão de probabilidade


In [8]:
ativos = ["PETR4", "VALE3", "BOVA11", "MGLU3", "CIEL3", "CSNA3", "COGN3", "IRBR3", "VIIA3", "B3SA3", "BBAS3", "WEGE3", 
          "USIM5", "BRFS3", "BBDC4", "ITSA4", "ABEV3", "JBSS3", "PRIO3"] #pode incluir mais ativos
ativos = [i + '.SA' for i in ativos]


## 3. Baixando dados

In [9]:
df = yf.download(ativos, start = inicio, end = hoje)['Adj Close']


[*********************100%***********************]  19 of 19 completed

2 Failed downloads:
- COGN3.SA: No data found for this date range, symbol may be delisted
- BRFS3.SA: No data found for this date range, symbol may be delisted


## 4. Tratamento os dados

In [13]:
df = df.dropna()
retornos = df.pct_change()
retornos = retornos.dropna()
#data_ref = datetime.date.today()-datetime.timedelta(days = 1) #usa o preço da data de ontem como referência 
#data_ref = str(data_ref)
preco_ref = df.iloc[-1]


## 5. Calculo do EWMA

In [14]:
length = len(retornos)
Lambda = 0.94
ser = np.arange(length-1,-1,-1)
exp = np.power(Lambda,ser)*(1-Lambda)
prod = pd.DataFrame()

for i in retornos:
  prod[i] = exp*retornos.loc[:,i]**2
  EWMA = np.sqrt(prod.sum())

## 6. Previsão para periodo

In [15]:
desvio = EWMA
media = retornos.mean()
medret = (((1+media)**(dias_venc))-1)
desvret = (desvio*np.sqrt(dias_venc))

## 7. Calculo dos preços de cada Desvio

In [16]:
desvio_05_p = round(preco_ref*(1+medret+0.5*desvret),2)
desvio_10_p = round(preco_ref*(1+medret+desvret),2)
desvio_15_p = round(preco_ref*(1+medret+1.5*desvret),2)
desvio_20_p = round(preco_ref*(1+medret+2*desvret),2)

desvio_05_n = round(preco_ref*(1+medret-0.5*desvret),2)
desvio_10_n = round(preco_ref*(1+medret-desvret),2)
desvio_15_n = round(preco_ref*(1+medret-1.5*desvret),2)
desvio_20_n = round(preco_ref*(1+medret-2*desvret),2)

## 8. Visualizando os valores dos desvios dos preços

In [17]:
desvios = pd.DataFrame([desvio_20_p, desvio_15_p, desvio_10_p, desvio_05_p, desvio_05_n, desvio_10_n, desvio_15_n, desvio_20_n])
desvios = desvios.rename({0: "2,0 desvios alta", 1: "1,5 desvios alta", 2: "1,0 desvios alta", 3: "0,5 desvios alta", 
                         7: "2,0 desvios baixa", 6: "1,5 desvios baixa", 5: "1,0 desvios baixa", 4: "0,5 desvios baixa"})
desvios

,ABEV3.SA,B3SA3.SA,BBAS3.SA,BBDC4.SA,BOVA11.SA,BRFS3.SA,CIEL3.SA,COGN3.SA,CSNA3.SA,IRBR3.SA,ITSA4.SA,JBSS3.SA,MGLU3.SA,PETR4.SA,PRIO3.SA,USIM5.SA,VALE3.SA,VIIA3.SA,WEGE3.SA
"2,0 desvios alta",15.51,15.72,47.42,16.63,122.06,10.07,6.48,2.70,21.20,40.92,9.60,24.74,6.35,32.87,50.95,9.57,106.86,3.47,42.36
"1,5 desvios alta",15.03,15.02,45.66,15.96,118.99,9.45,6.24,2.58,20.29,37.34,9.35,23.96,5.85,31.47,48.87,9.18,104.38,3.22,41.23
"1,0 desvios alta",14.55,14.32,43.89,15.30,115.92,8.83,5.99,2.46,19.38,33.75,9.09,23.17,5.36,30.07,46.80,8.79,101.91,2.97,40.11
"0,5 desvios alta",14.07,13.62,42.13,14.64,112.85,8.21,5.75,2.34,18.47,30.16,8.84,22.39,4.86,28.66,44.73,8.41,99.44,2.72,38.99
"0,5 desvios baixa",13.12,12.21,38.60,13.31,106.71,6.98,5.25,2.11,16.65,22.99,8.33,20.83,3.87,25.85,40.58,7.63,94.49,2.22,36.74
"1,0 desvios baixa",12.64,11.51,36.84,12.65,103.64,6.36,5.01,1.99,15.74,19.40,8.08,20.04,3.38,24.45,38.51,7.25,92.02,1.97,35.62
"1,5 desvios baixa",12.16,10.81,35.07,11.99,100.57,5.74,4.76,1.87,14.83,15.81,7.83,19.26,2.88,23.05,36.44,6.86,89.54,1.72,34.49
"2,0 desvios baixa",11.68,10.10,33.31,11.32,97.50,5.12,4.52,1.75,13.92,12.23,7.57,18.48,2.39,21.64,34.37,6.47,87.07,1.47,33.37


## 9. Visualização dos dados nos gráficos

In [18]:
ticker = "PETR4" #Colocar a ação que quer ver o gráfico
ticker = ticker + '.SA'

ativo = yf.download(ticker, start = inicio, end = hoje)


fig = make_subplots(rows=1, cols=1)
fig.add_trace(go.Scatter(name=ticker + ' Ajustado', x=ativo.index, y=ativo['Adj Close']),row=1,col=1)
fig.add_trace(go.Candlestick(name=ticker, x=ativo.index, close=ativo['Close'], open=ativo['Open'],high=ativo['High'], low=ativo['Low']))
fig.add_hline(y=desvios.loc["0,5 desvios alta",ticker], line = {'color': '#ee5253', 'dash':'dot'}, 
              annotation_text=(f'+0,5 desvio -> {desvios.loc["0,5 desvios alta",ticker]}'), annotation_position="top left")
fig.add_hline(y=desvios.loc["1,0 desvios alta",ticker], line = {'color': '#ee5253', 'dash':'dot'}, 
              annotation_text=(f'+1,0 desvio -> {desvios.loc["1,0 desvios alta",ticker]}'), annotation_position="top left")
fig.add_hline(y=desvios.loc["1,5 desvios alta",ticker], line = {'color': '#ee5253', 'dash':'dot'}, 
              annotation_text=(f'+1,5 desvio -> {desvios.loc["1,5 desvios alta",ticker]}'), annotation_position="top left")
fig.add_hline(y=desvios.loc["2,0 desvios alta",ticker], line = {'color': '#ee5253', 'dash':'dot'}, 
              annotation_text=(f'+2,0 desvio -> {desvios.loc["2,0 desvios alta",ticker]}'), annotation_position="top left")

fig.add_hline(y=desvios.loc["0,5 desvios baixa",ticker], line = {'color': '#ee5253', 'dash':'dot'}, 
              annotation_text=(f'-0,5 desvio -> {desvios.loc["0,5 desvios baixa",ticker]}'), annotation_position="bottom left")
fig.add_hline(y=desvios.loc["1,0 desvios baixa",ticker], line = {'color': '#ee5253', 'dash':'dot'}, 
              annotation_text=(f'-1,0 desvio -> {desvios.loc["1,0 desvios baixa",ticker]}'), annotation_position="bottom left")
fig.add_hline(y=desvios.loc["1,5 desvios baixa",ticker], line = {'color': '#ee5253', 'dash':'dot'}, 
              annotation_text=(f'-1,5 desvio -> {desvios.loc["1,5 desvios baixa",ticker]}'), annotation_position="bottom left")
fig.add_hline(y=desvios.loc["2,0 desvios baixa",ticker], line = {'color': '#ee5253', 'dash':'dot'}, 
              annotation_text=(f'-2,0 desvio -> {desvios.loc["2,0 desvios baixa",ticker]}'), annotation_position="bottom left")


fig.update_layout(xaxis_rangeslider_visible=False)
fig.show()

[*********************100%***********************]  1 of 1 completed
